# E2Style: Improve the Efficiency and Effectiveness of StyleGAN Inversion
---



## Resources

- 📃 [Paper](https://wty-ustc.github.io/inversion/paper/E2Style.pdf)
- 📚 [Project Page](https://wty-ustc.github.io/inversion)
- 🎬 [Examples](https://youtu.be/gJwFgdRHK0M)
- 💻 [Code](https://github.com/wty-ustc/e2style)

## Abstract

[Abstract](https://wty-ustc.github.io/inversion/paper/E2Style.pdf)—*This paper studies the problem of StyleGAN inversion, which plays an essential role in enabling the pretrained
StyleGAN to be used for real image editing tasks. The goal of
StyleGAN inversion is to find the exact latent code of the given
image in the latent space of StyleGAN. This problem has a high
demand for quality and efficiency. Existing optimization-based
methods can produce high-quality results, but the optimization
often takes a long time. On the contrary, forward-based methods
are usually faster but the quality of their results is inferior. In
this paper, we present a new feed-forward network “E2Style”
for StyleGAN inversion, with significant improvement in terms
of efficiency and effectiveness. In our inversion network, we
introduce: 1) a shallower backbone with multiple efficient heads
across scales; 2) multi-layer identity loss and multi-layer face
parsing loss to the loss function; and 3) multi-stage refinement.
Combining these designs together forms an effective and efficient method that exploits all benefits of optimization-based
and forward-based methods. Quantitative and qualitative results
show that our E2Style performs better than existing forwardbased methods and comparably to state-of-the-art optimizationbased methods while maintaining the high efficiency as well
as forward-based methods. Moreover, a number of real image
editing applications demonstrate the efficacy of our E2Style. Our
code is available at* https://github.com/wty-ustc/e2style


## Authors

Tianyi Wei<sup>1</sup>,
Dongdong Chen<sup>2</sup>,
Wenbo Zhou<sup>1</sup>,
Jing Liao<sup>3</sup>,
Weiming Zhang<sup>1</sup>, 
Lu Yuan<sup>2</sup>, 
Gang Hua<sup>4</sup>, 
Nenghai Yu<sup>1</sup> <br>
<sup>1</sup>*University of Science and Technology of China,*<br>
<sup>2</sup>*Microsoft Cloud AI*<br>
<sup>3</sup>*City University of Hong Kong,*<br>
<sup>4</sup>*Wormpex AI Research*

## Citation

### Plain Text



```
T. Wei et al., "E2Style: Improve the Efficiency and Effectiveness of StyleGAN Inversion," in IEEE Transactions on Image Processing, vol. 31, pp. 3267-3280, 2022, doi: 10.1109/TIP.2022.3167305.
```



### BibTex



```
@ARTICLE{9760266,
  author={Wei, Tianyi and Chen, Dongdong and Zhou, Wenbo and Liao, Jing and Zhang, Weiming and Yuan, Lu and Hua, Gang and Yu, Nenghai},
  journal={IEEE Transactions on Image Processing}, 
  title={E2Style: Improve the Efficiency and Effectiveness of StyleGAN Inversion}, 
  year={2022},
  volume={31},
  number={},
  pages={3267-3280},
  doi={10.1109/TIP.2022.3167305}}
```



# Set up the notebook

In [ ]:
# @title ## Mount your Google Drive at `/content/gdrive`
from google.colab import drive
drive.mount('/content/gdrive')

## Clone the [`e2style`](https://github.com/wty-ustc/e2style) repository

In [ ]:
!git clone https://github.com/wty-ustc/e2style.git
%cd e2style

## Install requirements

In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install ninja

## Download the models

In [ ]:
# @markdown Download the following models: [`stylegan2-ffhq-config-f.pt`](https://drive.google.com/file/d/1pts5tkfAcWrg4TpLDu6ILF5wHID32Nzm/view?usp=sharing )
# @markdown & [`model_ir_se50.pth`](https://drive.google.com/file/d/1FS2V756j-4kWduGxfir55cMni5mZvBTv/view)
!gdown --fuzzy https://drive.google.com/file/d/1pts5tkfAcWrg4TpLDu6ILF5wHID32Nzm/view?usp=sharing 
!gdown --fuzzy https://drive.google.com/file/d/1FS2V756j-4kWduGxfir55cMni5mZvBTv/view?usp=sharing

In [ ]:
# @markdown Move the models to the `pretrained_models` directory
!cp stylegan2-ffhq-config-f.pt pretrained_models/
!cp model_ir_se50.pth pretrained_models/

# Training

## Prepare the datasets

### Set up the FFQH dataset

In [ ]:
# @markdown Prepare a directory for the dataset
ffhq_dir = "/content/e2style/data/ffhq"
!mkdir -p $ffhq_dir

In [ ]:
# @markdown Copy the dataset from your Google Drive
!cp -r /content/gdrive/MyDrive/datasets/ffhq/images1024x1024 $ffhq_dir

### Set up the CelebA-HQ dataset

In [ ]:
# @markdown Prepare a directory for the dataset
celeba_dir = "/content/e2style/data/celeba"
!mkdir -p $celeba_dir

In [ ]:
# @title #### Download the [CelebAMask-HQ](https://github.com/switchablenorms/CelebAMask-HQ) dataset

!gdown --fuzzy https://drive.google.com/file/d/1badu11NqxGf6qM3PTTooQDJvQbejgbTv/view
!unzip CelebAMask-HQ.zip -d $celeba_dir

#### Set up the CelebA-HQ test split

In [ ]:
# @title ##### Download CelebA dataset's official split configuration
!gdown --fuzzy https://drive.google.com/file/d/0B7EVK8r0v71pY0NSMzRuSXJEVkk/view?resourcekey=0-i4TGCi_51OtQ5K9FSp4EDg -O $celeba_dir/list_eval_partition.txt

##### Map the local CelebA-HQ dataset according to the split configuration

In [ ]:
with open(f"{celeba_dir}/list_eval_partition.txt", "r") as f:
    text = f.read()

list_eval_partition = [l.split(" ") for l in text.split("\n")]

celeba_test_files = []
for l in list_eval_partition:
    if len(l) < 2:
        continue
    f, s = l

    if int(s) == 2:
        celeba_test_files.append(f)

In [ ]:
with open(f"{celeba_dir}/CelebAMask-HQ/CelebA-HQ-to-CelebA-mapping.txt", "r") as f:
    text = f.read()

lines = text.split("\n")
celeba_mapping = [l.split() for l in lines][1:]

In [ ]:
import glob

celebahq_files = glob.glob(f"{celeba_dir}/CelebAMask-HQ/CelebA-HQ-img/*.jpg")

celebahq_test_files = []
for f in celebahq_files:
    name = f.split("/")[-1]
    idx = int(name.split(".")[0])
    mapped = celeba_mapping[idx][-1]
    if mapped in celeba_test_files:
        celebahq_test_files.append(name)

In [ ]:
from tqdm import tqdm

celeba_test_dir = f"{celeba_dir}/test"
!mkdir -p $celeba_test_dir

for f in tqdm(celebahq_test_files):
    !cp $celeba_dir/CelebAMask-HQ/CelebA-HQ-img/$f $celeba_test_dir

## Setup the dataset paths

In [ ]:
%env CELEBA_DIR=$celeba_dir
%env FFHQ_DIR=$ffhq_dir

In [ ]:
%%writefile /content/e2style/configs/paths_config.py

import os

CELEBA_DIR = os.environ["CELEBA_DIR"]
FFHQ_DIR = os.environ["FFHQ_DIR"]

dataset_paths = {
	'celeba_train': '',
	'celeba_test': f'{CELEBA_DIR}/test',
	'celeba_train_4seg': '',
	'celeba_test_4seg': '',	
	'celeba_train_sketch': '',
	'celeba_test_sketch': '',
	'celeba_train_segmentation': '',
	'celeba_test_segmentation': '',
	'ffhq': f'{FFHQ_DIR}/images1024x1024',
}

model_paths = {
	'stylegan_ffhq': 'pretrained_models/stylegan2-ffhq-config-f.pt',
	'ir_se50': 'pretrained_models/model_ir_se50.pth',
	'parsing_net': 'pretrained_models/parsing.pth',
	'circular_face': 'pretrained_models/CurricularFace_Backbone.pth',
	'mtcnn_pnet': 'pretrained_models/mtcnn/pnet.npy',
	'mtcnn_rnet': 'pretrained_models/mtcnn/rnet.npy',
	'mtcnn_onet': 'pretrained_models/mtcnn/onet.npy',
	'shape_predictor': 'pretrained_models/shape_predictor_68_face_landmarks.dat'
}

## Train!

In [ ]:
import random
rnd = random.randint(0, 1_000_000)
print("exp_dir:", f"exp/{rnd}")

%cd /content/e2style
!python scripts/train.py \
    --dataset_type=ffhq_encode \
    --exp_dir=./exp/$rnd \
    --workers=4 \
    --batch_size=4 \
    --test_batch_size=4 \
    --test_workers=4 \
    --val_interval=5000 \
    --save_interval=5000 \
    --start_from_latent_avg \
    --lpips_lambda=0.8 \
    --l2_lambda=1 \
    --id_lambda=0.5 \
    --parse_lambda=1 \
    --training_stage=1

# Inference

In [ ]:
# @markdown Move into the `e2style` repository root
%cd /content/e2style

In [ ]:
# @markdown Download a pretrained model
!gdown --fuzzy https://drive.google.com/file/d/1CzMDA88GJgVzc5JxKt3-l504a7TuSw5j/view?usp=sharing

In [ ]:
# @markdown Prepare some test data
!mkdir test_data
!cp $ffhq_dir/00000/00000.png test_data/
!cp $ffhq_dir/00000/00001.png test_data/

In [ ]:
import random
rnd = random.randint(0, 1_000_000)
print("exp_dir:", f"exp/{rnd}")

!python scripts/inference.py \
    --exp_dir=./inf/$rnd \
    --checkpoint_path=inversion.pt \
    --data_path=./test_data \
    --test_batch_size=1 \
    --test_workers=4 \
    --stage=1 \
    --save_inverted_codes \
    --couple_outputs \
    --resize_outputs